In [1]:
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from oauth2client.tools import argparser
from itertools import chain
from urllib import request
import pandas as pd
import numpy as np
import json
import os

In [2]:
IS_JUPYTER = True
ROOT_PATH  = '/home/jovyan/project' if IS_JUPYTER else '/config/workspace/project'

JSON_PATH        = f'{ROOT_PATH}/utils/keys'
JSON_BACKUP_PATH = f'{ROOT_PATH}/BACKUPS/keys'

In [3]:
load_json = lambda json_path: json.loads(open(json_path, 'r').read())
save_json = lambda data, json_path: json.dump(data, open(json_path, 'w').write())

def repair_keys(json_path):
    try: keys = load_json(f'{json_path}/keys.json')
        
    except Exception as e:
        print(f'[ERR.K-002] json 파일이 깨져 열 수 없습니다. {e}')
        text = open(f'{JSON_BACKUP_PATH}/keys.txt', 'r').read().split('\n')
        keys = {platform : key 
                for platform, key in zip(['youtube', 'steam'], text)}
        
        save_paths = [JSON_PATH, JSON_BACKUP_PATH]
        for save_path in save_paths:
            os.makedirs(save_path, exist_ok = True)
            save_json(keys, f'{save_path}/keys.json')
        
    finally: return keys



if os.path.isfile(f'{JSON_PATH}/keys.json'):
    keys = repair_keys(JSON_PATH)
        
else:
    print(f'[ERR.K-001] json 파일이 존재하지 않아 백업 데이터를 로딩합니다.')
    keys = repair_keys(JSON_BACKUP_PATH)

In [4]:
## youtube api 사용할 수 있게 해주는 함수
def use_api():
    ## 개인 youtube api키 받고 사용하기 
    API_KEY          = keys['youtube']
    API_SERVICE_NAME = 'youtube'
    API_VERSION      = 'v3'
    
    return build(API_SERVICE_NAME, API_VERSION, developerKey = API_KEY)

In [5]:
from pprint import pprint

## 채널부터 시작해 영상 목록까지 가져오는 함수
def search_datas(query, max_result = 30):
    youtube = use_api()
    
    ## query에 해당하는 채널 이름 찾기
    ## max_result의 갯수 만큼 채널을 찾아 가져옴.
    result  = youtube.search().list(
                q          = query,
                part       = 'snippet',
                maxResults = max_result
            ).execute()
    
    ## youtube api에서 가져온 데이터가 json 데이터에 좀 복잡한 구조로 되어있는데,
    ## 채널 id는 channel_id 변수처럼 가져올 수 있다.
    channel_id = result['items'][0]['id']['channelId']
    
    ## 채널 내 플레이리스트 id 목록 가져옴.
    playlists  = search_playlists(youtube, channel_id)
    
    ## 영상 업로드 날짜, 제목, id, 썸네일 링크를 저장
    dates, titles, ids, thumbnails = [], [], [], []
    for playlist in playlists: 
        (d, t, i, th) = search_videos(youtube, playlist)
        dates      = list(chain(dates    , d))
        titles     = list(chain(titles   , t))
        ids        = list(chain(ids      , i))
        thumbnails = list(chain(thumbnails, th))
    
    ## 저장한 영상정보 데이터를 DataFrame으로 변환
    df         = pd.DataFrame([dates, titles, ids, thumbnails]).T
    df.columns = ['dates', 'titles', 'ids', 'thumbnails']
    return df
    
##채널 내 플레이리스트의 id 가져오는 함수
def search_playlists(youtube, id, max_result = 100):
    ## channelId에 해당하는 플레이리스트 찾기
    ## max_result의 갯수 만큼의 플레이리스트를 찾아 가져옴.
    result = youtube.playlists().list(
                channelId  = id,
                part       = 'snippet',
                maxResults = max_result
            ).execute()
    
    ## 플레이리스트 id 저장 
    ids = []
    for playlist in result['items']: 
        title = playlist['snippet']['localized']['title']
        if '2022년 침착맨 정주행' in title: ids.append(playlist['id'])
    return ids

## 채널의 플레이리스트에 있는 영상 정보 가져오는 함수
def search_videos(youtube, id, max_result = 200):
    ## playlistId에 해당하는 영상 찾기
    ## max_result의 갯수 만큼 영상을 찾아 가져옴.
    result = youtube.playlistItems().list(
                playlistId = id,
                part       = 'snippet',
                maxResults = max_result
            ).execute()
    titles, ids, thumbnails, dates = [], [], [],[]
    
    for video in result['items']:
        snippet = video['snippet']
        
        ## 비공개 동영상인 경우에 그냥 넘기기 
        if 'private' in snippet['title'].lower(): continue
        
        titles.append(snippet['title'])
        ids.append(snippet['resourceId']['videoId'])
        dates.append(snippet['publishedAt'])
        
        ## 썸네일에 medium 이미지가 없는 경우 default 이미지로 가져오기
        try: thumbnails.append(snippet['thumbnails']['medium']['url'])
        except: thumbnails.append(snippet['thumbnails']['default']['url'])
        
    return (dates, titles, ids, thumbnails)

In [6]:
df = search_datas('침착맨', max_result = 1)

In [7]:
df

,dates,titles,ids,thumbnails
0,2022-05-22T12:56:52Z,환타로 만든 뱅쇼 vs 쌍화탕으로 만든 모주,0pw5gBIAWQM,https://i.ytimg.com/vi/0pw5gBIAWQM/mqdefault.jpg
1,2022-05-22T12:56:52Z,2021년은 어땠나? 2022년은 어떨까?,rwYoIzl5o9U,https://i.ytimg.com/vi/rwYoIzl5o9U/mqdefault.jpg
2,2022-05-22T12:56:52Z,치킨계의 냉전시대,-YZ_z_TRBnY,https://i.ytimg.com/vi/-YZ_z_TRBnY/mqdefault.jpg
3,2022-05-22T12:56:52Z,90년대 애니 여캐 둘러보기,IcTjuJGoPbQ,https://i.ytimg.com/vi/IcTjuJGoPbQ/mqdefault.jpg
4,2022-05-22T12:56:52Z,저가요 거위인데요,bDKA_p8jhSw,https://i.ytimg.com/vi/bDKA_p8jhSw/mqdefault.jpg
5,2022-05-22T12:56:52Z,꽁치가처넌의 멸망의 노래!,qMc79yi2N4A,https://i.ytimg.com/vi/qMc79yi2N4A/mqdefault.jpg
6,2022-05-22T12:56:52Z,침착맨과 인기웹툰 '나 혼자만 레벨업' 같이 보기,Z__KMr6QLEQ,https://i.ytimg.com/vi/Z__KMr6QLEQ/mqdefault.jpg
7,2022-05-22T12:56:52Z,맞기 싫은 몽둥이 월드컵,V0pNWjWlazo,https://i.ytimg.com/vi/V0pNWjWlazo/mqdefault.jpg
8,2022-05-22T12:56:52Z,"새해 금연합니다, 현상금 1000만원",yJI9iCKlD9A,https://i.ytimg.com/vi/yJI9iCKlD9A/mqdefault.jpg
9,2022-05-22T12:56:52Z,특이한 벌레와 새 이야기,5T_4pnS6nxA,https://i.ytimg.com/vi/5T_4pnS6nxA/mqdefault.jpg


In [8]:
df.to_csv('chim_videos.csv', index = False)

In [9]:
## 채널부터 시작해 영상 목록까지 가져오는 함수
def search_datas(query, max_result = 30):
    youtube = use_api()
    
    ## query에 해당하는 채널 이름 찾기
    ## max_result의 갯수 만큼 채널을 찾아 가져옴.
    result  = youtube.search().list(
                q          = query,
                part       = 'snippet',
                maxResults = max_result
            ).execute()
    
    ## youtube api에서 가져온 데이터가 json 데이터에 좀 복잡한 구조로 되어있는데,
    ## 채널 id는 channel_id 변수처럼 가져올 수 있다.
    channel_id = result['items'][0]['id']['channelId']
    playlists  = search_playlists(youtube, channel_id)
    
    return playlists
    
##채널 내 플레이리스트의 id 가져오는 함수
def search_playlists(youtube, id, max_result = 100):
    ## channelId에 해당하는 플레이리스트 찾기
    ## max_result의 갯수 만큼의 플레이리스트를 찾아 가져옴.
    result = youtube.playlists().list(
                channelId  = id,
                part       = 'snippet',
                maxResults = max_result
            ).execute()
    
    ## 플레이리스트 id 저장 
    ids = []
    
    for playlist in result['items']: 
        title = playlist['snippet']['localized']['title']
        if '침착맨 정주행' in title: ids.append(playlist['id'])
    return ids

df = search_datas('침착맨', max_result = 1)
df

['PLif_jr7pPZAB6YJFfAg8riV6NtIOqez6j',
 'PLif_jr7pPZABZsONyeNz1JvpAOxPi8FIs',
 'PLif_jr7pPZADliLF8xwsm7BJHUqq_CUKo',
 'PLif_jr7pPZADJ2MJ-iHlgLijM6xkCaFlo',
 'PLif_jr7pPZAD8EJtkFBBctqm5L38JqtGb']

## 전체 영상 가져오기
[[여기 확인해보기]](https://velog.io/@keemsw/%EC%9C%A0%ED%8A%9C%EB%B8%8C-API%EB%A5%BC-%ED%99%9C%EC%9A%A9%ED%95%98%EC%97%AC-%EB%8F%99%EC%98%81%EC%83%81-%EB%A6%AC%EC%8A%A4%ED%8A%B8%EC%97%85-%ED%95%98%EA%B8%B0)

In [10]:
from collections import namedtuple
import pandas as pd

csv = pd.read_csv('chim_videos.csv')
vid = namedtuple('videos', ['dates', 'titles', 'ids', 'thumbnails'])

datas = zip(*[csv[c] for c in csv.columns])
datas = [vid._make(data) for data in datas]

In [11]:
datas

[videos(dates='2022-05-22T12:56:52Z', titles='환타로 만든 뱅쇼 vs 쌍화탕으로 만든 모주', ids='0pw5gBIAWQM', thumbnails='https://i.ytimg.com/vi/0pw5gBIAWQM/mqdefault.jpg'),
 videos(dates='2022-05-22T12:56:52Z', titles='2021년은 어땠나? 2022년은 어떨까?', ids='rwYoIzl5o9U', thumbnails='https://i.ytimg.com/vi/rwYoIzl5o9U/mqdefault.jpg'),
 videos(dates='2022-05-22T12:56:52Z', titles='치킨계의 냉전시대', ids='-YZ_z_TRBnY', thumbnails='https://i.ytimg.com/vi/-YZ_z_TRBnY/mqdefault.jpg'),
 videos(dates='2022-05-22T12:56:52Z', titles='90년대 애니 여캐 둘러보기', ids='IcTjuJGoPbQ', thumbnails='https://i.ytimg.com/vi/IcTjuJGoPbQ/mqdefault.jpg'),
 videos(dates='2022-05-22T12:56:52Z', titles='저가요 거위인데요', ids='bDKA_p8jhSw', thumbnails='https://i.ytimg.com/vi/bDKA_p8jhSw/mqdefault.jpg'),
 videos(dates='2022-05-22T12:56:52Z', titles='꽁치가처넌의 멸망의 노래!', ids='qMc79yi2N4A', thumbnails='https://i.ytimg.com/vi/qMc79yi2N4A/mqdefault.jpg'),
 videos(dates='2022-05-22T12:56:52Z', titles="침착맨과 인기웹툰 '나 혼자만 레벨업' 같이 보기", ids='Z__KMr6QLEQ', thumbnails='https:/